In [4]:
import httplib2
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.credentials import Credentials
from datetime import datetime
import json
from pathlib import Path
import xlsxwriter
import json
from pathlib import Path
from datetime import date, timedelta

In [7]:
import Core.GoogleSheetsService as givno

In [12]:
givno.updateUniqueMessages()

[2022-11-04 13:40:18,516: DEBUG] GoogleSheetsService.py	-> updateUniqueMessages[59] 	[[], ['unknownState', 'Кажется, сейчас я тебя не понимаю...'], ['menuButtonReturnToMainMenu', 'Главное меню'], [], ['menuButtonExercises', 'Перейти к упражнениям'], ['menuButtonRandomNews', 'Я не хочу делать упражнения, просто хочу переключиться'], ['menuButtonRelax', 'Мне просто нужно отдохнуть'], ['menuButtonAdmin', 'Админ'], ['mainMenuText', 'Твое состояние сейчас определяется твоими эмоциями и мыслями.\n\nЗабот поможет тебе разобраться хитросплетении внутренних переживаний и успокоить их.'], [], ['adminMenuText', 'Вы в меню администратора'], ['adminMenuButtonLoadData', 'Выгрузить данные'], ['adminMenuButtonReloadData', 'Стянуть новые тексты из гугл таблицы'], [], ['randomNewsStartText', 'Сейчас расскажу тебе что-нибудь интересное'], ['randomNewsButtonNews', 'Хочу еще'], ['randomNewsAllNewsWasShown', 'Кажется, мне больше нечего тебе показать\nЗагляни сюда позже'], [], ['exercisesStartText', 'Выбери,

In [11]:
for kek in range(0️⃣,🔟):
    print(kek)

SyntaxError: invalid character in identifier (<ipython-input-11-efae2994d901>, line 1)